In [172]:
import numpy as np

In [7]:
c = np.array([-3, 8, 0, 0])
c = np.transpose(c)
b = np.array([12, 6])
A = np.array([[4, 2, 1, 0],[2, 3, 0, 1]])

c_split = np.array_split(c, 2)
c_b = c_split[1]
c_n = c_split[0]
print(c_b, c_n)
minimize = True

[0 0] [-3  8]


In [22]:
import numpy as np

c = np.array([3, 2, -1, -2, 1, 2, -1, 3, 4, -3, 0, 0, 0, 0, 0])
b = np.array([80, 50, 40, 90, 50])
A = np.array([[2, 1, 3, 1, 2, 1, 4, 1, -2, 3, 1, 0, 0, 0, 0],
              [1, -4, 1, 2, 3, 1, -1, 4, 1, 2, 0, 1, 0, 0, 0],
              [3, 2, -2, -1, 1, 3, 2, 1, 1, 1, 0, 0, 1, 0, 0],
              [2, 3, 1, 1, 1, 1, 1, 1, 3, 2, 0, 0, 0, 1, 0],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1]])
slack_vars = 5
c_b = c[(-1*slack_vars):]
c_n = c[:(-1*slack_vars)]
B = A[:, (-1*slack_vars):]
N = A[:, :(-1*slack_vars)]
print("Basic coefficient vector:", c_b)
print("Non-basic coefficient vector:", c_n)
print("B Matrix: \n", B)
print("N Matrix: \n", N)

print(c_b.dot(np.linalg.inv(B).dot(N)) - c_n)

[0 0 0 0 0] [ 3  2 -1 -2  1  2 -1  3  4 -3]
B Matrix: 
 [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
N Matrix: 
 [[ 2  1  3  1  2  1  4  1 -2  3]
 [ 1 -4  1  2  3  1 -1  4  1  2]
 [ 3  2 -2 -1  1  3  2  1  1  1]
 [ 2  3  1  1  1  1  1  1  3  2]
 [ 1  1  1  1  1  1  1  1  1  1]]
[-3. -2.  1.  2. -1. -2.  1. -3. -4.  3.]


In [174]:
# Iteration 1
if minimize == True:
    A_split = np.hsplit(A, 2)
    B = A_split[1]
    N = A_split[0]

    # determining var entering basic set B
    res_mat = c_b.dot(np.linalg.inv(B).dot(N)) - c_n

    max_index = max(res_mat)
    res = [i for i, j in enumerate(res_mat) if j == max_index]

    print(res_mat)

    # determining var leaving non-basic set N

    a_entering = [row[res[0]] for row in A]

    # B(-1)*b/B(-1)*a 
    res_mat2 = np.divide(np.linalg.inv(B).dot(b), (np.linalg.inv(B).dot(a_entering)))
    print((np.linalg.inv(B).dot(a_entering)))

    max_index2 = max(res_mat2)
    res2 = [i for i, j in enumerate(res_mat2) if j == max_index2]

    enter_col = np.copy(N[:,res[0]])
    exit_col = np.copy(B[:,res2[0]])
    enter_cB = np.copy(c_n[res[0]])
    enter_cN = np.copy(c_b[res2[0]])
    #if cost vectors are fine after iteration 1, compute z = cB*xB
    if all(i <= 0 for i in res_mat) == True:
        x_b = np.linalg.inv(B).dot(b)
        print("The optimal solution is:", c_b.dot(x_b))

elif minimize == False:
    A_split = np.hsplit(A, 2)
    B = A_split[1]
    N = A_split[0]
    print(B)

    # determining var entering basic set B
    res_mat = c_b.dot(np.linalg.inv(B).dot(N)) - c_n

    min_index = min(res_mat)
    res = [i for i, j in enumerate(res_mat) if j == min_index]

    print("res_mat =", res_mat)
    print("The basic variable leaving is index:", res[0] + 1)
    # determining var leaving non-basic set N

    a_entering = [row[res[0]] for row in A]

    # B(-1)*b/B(-1)*a 
    res_mat2 = np.divide(np.linalg.inv(B).dot(b), (np.linalg.inv(B).dot(a_entering)))

    min_index2 = min(res_mat2)
    res2 = [i for i, j in enumerate(res_mat2) if j == min_index2]
    print(c_b[res[0]])

    enter_col = np.copy(N[:,res[0]])
    exit_col = np.copy(B[:,res2[0]])
    enter_cB = np.copy(c_n[res[0]])
    enter_cN = np.copy(c_b[res2[0]])
    print(res_mat2)
    print("The non-basic variable entering is index:", res2[0] + 1)
    #if cost vectors are fine after iteration 1, compute z = cB*xB
    if all(i >= 0 for i in res_mat) == True:
        x_b = np.linalg.inv(B).dot(b)
        print("The optimal solution is:", c_b.dot(x_b))

[ 3. -8.]
[4. 2.]


In [175]:
B[:,res2[0]] = enter_col
N[:,res[0]] = exit_col
c_b[res2[0]] = enter_cB
c_n[res[0]] = enter_cN

In [176]:
# Following iterations

# if cB*B-1*N - cN values are non-positive, solution is optimal:
if minimize == True:
    while all(i <= 0 for i in res_mat) == False:        
        # determining var entering basic set B
        res_mat = c_b.dot(np.linalg.inv(B)).dot(N) - c_n
        print("res_mat =", res_mat)

        max_index = max(res_mat)
        res = [i for i, j in enumerate(res_mat) if j == max_index]

        # determining var leaving non-basic set N

        a_entering = [row[res[0]] for row in A]

        # B(-1)*b/B(-1)*a 
        res_mat2 = np.divide(np.linalg.inv(B).dot(b), (np.linalg.inv(B).dot(a_entering)))
        print(res_mat2)

        max_index2 = max(res_mat2)
        res2 = [i for i, j in enumerate(res_mat2) if j == max_index2]

        enter_col = np.copy(N[:,res[0]])
        exit_col = np.copy(B[:,res2[0]])
        enter_cB = np.copy(c_n[res[0]])
        enter_cN = np.copy(c_b[res2[0]])

        if (all(i<= 0 for i in res_mat) == True):
            continue
        else:
            B[:,res2[0]] = enter_col
            N[:,res[0]] = exit_col
            c_b[res2[0]] = enter_cB
            c_n[res[0]] = enter_cN
    if all(i <= 0 for i in res_mat) == True:
        x_b = np.linalg.inv(B).dot(b)
        print("The optimal solution is:", c_b.dot(x_b))
# if cB*B-1*N - cN values are non-negative, solution is optimal:
elif minimize == False: # if the question calls for maximization
    while all(i>= 0 for i in res_mat) == False:
        B[:,res2[0]] = enter_col
        N[:,res[0]] = exit_col
        c_b[res2[0]] = enter_cB
        c_n[res[0]] = enter_cN

        res_mat = c_b.dot(np.linalg.inv(B)).dot(N) - c_n

        min_index = min(res_mat)
        res = [i for i, j in enumerate(res_mat) if j == min_index]

        print("res_mat =", res_mat)

        # determining var leaving non-basic set N

        a_entering = [row[res[0]] for row in A]

        # B(-1)*b/B(-1)*a 
        res_mat2 = np.divide(np.linalg.inv(B).dot(b), (np.linalg.inv(B).dot(a_entering)))

        min_index2 = min(res_mat2)
        res2 = [i for i, j in enumerate(res_mat2) if j == min_index2]

        enter_col = np.copy(N[:,res[0]])
        exit_col = np.copy(B[:,res2[0]])
        enter_cB = np.copy(c_n[res[0]])
        enter_cN = np.copy(c_b[res2[0]])

        if (all(i>= 0 for i in res_mat) == True):
            continue
        else:
            B[:,res2[0]] = enter_col
            N[:,res[0]] = exit_col
            c_b[res2[0]] = enter_cB
            c_n[res[0]] = enter_cN

        print("B matrix is", B)
    if all(i >= 0 for i in res_mat) == True:
        x_b = np.linalg.inv(B).dot(b)
        print("The optimal solution is:", c_b.dot(x_b))



res_mat = [-0.75 -9.5 ]
[12. -0.]
The optimal solution is: -9.0
